# Setup infrastructure using docker compose

In [3]:
from jinja2 import Environment, Template
import os

In [4]:
# set env vars

current_path = ".."
base_data_path = f'{current_path}/data'

server_port = 5555
test_data_path = base_data_path + "/test_data"
shared_data_path = base_data_path + "/shared_data"
value_key = "POWER_PRODUCTION"

In [20]:
# ids not to use (broken data)
# site_0003
# site_0004
# site_0014

site_ids = [
    "site_0001",
    "site_0002",
    "site_0003",
    "site_0004",
    "site_0005",
    "site_0006",
    "site_0007",
    "site_0008",
    "site_0009",
    "site_0010",
    "site_0011",
    "site_0012",
    "site_0013",
    "site_0014",
    "site_0015",
    "site_0016",
    "site_0017",
    "site_0018",
    "site_0019",
    "site_0020",
    "site_0021",
    "site_0022",
    "site_0023",
    "site_0024",
    "site_0025",
    "site_0026",
    "site_0027",
    "site_0028",
    "site_0029",
    "site_0030"
]


site_ids_training = [
    "site_0030",
    "site_0027",
    "site_0022",
    "site_0024",
    "site_0009",
    "site_0005",
    "site_0018",
    "site_0006",
    "site_0017",
    "site_0028",
    "site_0012"
]

site_ids_prediction = [
    "site_0026", # orientation 1, independent population
    "site_0021", # orientation 4, independent population
    "site_0030",
    "site_0027",
    "site_0022",
    "site_0024",
    "site_0009",
    "site_0005",
    "site_0018",
    "site_0006",
    "site_0017",
    "site_0012"
]

site_ids_training = [
    "site_0026", # orientation 1, independent population
    "site_0021", # orientation 4, independent population
]

site_ids_prediction = [
    "site_0026", # orientation 1, independent population
    "site_0021", # orientation 4, independent population
]


In [21]:
# docker-compose.yml

# Define Docker Compose template as a string
docker_compose_server_template = """
version: '3'
services:
  federated-server:
    container_name: federated-server
    build:
      context: c
      dockerfile: docker/server/Dockerfile
    environment:
      - SERVER_PORT={{ server_port }}
      - SITE_INFO_PATH=/test_data/site_info.csv
      - DATA_PATH=/data
    ports:
      - "{{ server_port }}:{{ server_port }}"
    volumes:
      - server-data:/data
      - {{ test_data_path }}/site_info.csv:/test_data/site_info.csv:ro
    networks:
      - federated-network

volumes:
  server-data:
    external: false

networks:
  federated-network:
    driver: bridge
    external: true

"""

docker_compose_client_template = """
version: '3'
services:
  {% for site_id in site_ids %}
  federated-client-{{ site_id }}:
    container_name: federated-client-{{ site_id }}
    build:
      context: ../app
      dockerfile: docker/client/Dockerfile
    environment:
      - SITE_ID={{ site_id }}
      - SERVER_ADDRESS=federated-server
      - SERVER_PORT={{ server_port }}
      - TEST_DATA_PATH=/test_data
      - SHARED_DATA_PATH=/shared_data
      - DATA_PATH=/data
      - VALUE_KEY={{ value_key }}
      - RUN_TRAINING={{ run_training }}
      - RUN_PREDICTION={{ run_prediction }}
    volumes:
      - client-{{ site_id }}-data:/data
      - {{ test_data_path + '/' + site_id }}:/test_data:ro
      - {{ shared_data_path }}:/shared_data
    networks:
      - federated-network
  {% endfor %}

volumes:
  {% for site_id in site_ids %}client-{{ site_id }}-data:
    external: false
  {% endfor %}

networks:
  federated-network:
    driver: bridge
    external: true

"""

docker_compose_client_centralized_template = """
version: '3'
services:
  federated-client-centralized:
    container_name: federated-client-centralized
    build:
      context: app
      dockerfile: docker/client-centralized/Dockerfile
    environment:
      - SITE_IDS={{ site_ids }}
      - TEST_DATA_PATH=/test_data
      - SHARED_DATA_PATH=/shared_data
      - DATA_PATH=/data
      - VALUE_KEY={{ value_key }}
      - RUN_TRAINING={{ run_training }}
      - RUN_PREDICTION={{ run_prediction }}
    volumes:
      - client-centralized-data:/data
      - {{ test_data_path }}:/test_data:ro
      - {{ shared_data_path }}:/shared_data

volumes:
  client-centralized-data:
    external: false

"""

In [22]:
# Create a Jinja2 template from the string
template_server = Template(docker_compose_server_template)
template_client = Template(docker_compose_client_template)
template_centralized_client = Template(docker_compose_client_centralized_template)

# Render the template with variables
output_server = template_server.render(server_port=server_port,
                                test_data_path=test_data_path)

output_client_training = template_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path + '/federated/training', 
                         value_key=value_key, 
                         run_training=True,
                         run_prediction=False,
                         site_ids=site_ids_training)

output_centralized_client_training = template_centralized_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path + '/centralized/training', 
                         value_key=value_key, 
                         run_training=True,
                         run_prediction=False,
                         site_ids=','.join(site_ids_training))

output_client_prediction = template_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path + '/federated/prediction', 
                         value_key=value_key, 
                         run_training=False,
                         run_prediction=True,
                         site_ids=site_ids_prediction)

output_centralized_client_prediction = template_centralized_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path + '/centralized/prediction', 
                         value_key=value_key, 
                         run_training=False,
                         run_prediction=True,
                         site_ids=','.join(site_ids_prediction))

if not os.path.exists('../test'):
    os.makedirs('../test')

with open('../test/docker-compose.server.yml', 'w') as file:
    file.write(output_server)

with open('../test/docker-compose.client-prediction.yml', 'w') as file:
    file.write(output_client_prediction)

with open('../test/docker-compose.centralized-client-prediction.yml', 'w') as file:
    file.write(output_centralized_client_prediction)

with open('../test/docker-compose.client-training.yml', 'w') as file:
    file.write(output_client_training)

with open('../test/docker-compose.centralized-client-training.yml', 'w') as file:
    file.write(output_centralized_client_training)